In [ ]:
!pip install osmnx

In [5]:
import osmnx as ox
import sys
from queue import PriorityQueue
import folium

# 춘천의 도로 네트워크 데이터를 가져오기
location = "Chuncheon, South Korea"
G = ox.graph_from_place(location, network_type="drive")

# 인접 리스트 형태로 그래프 변환
V = len(G.nodes)
node_mapping = {node: idx for idx, node in enumerate(G.nodes)}
reverse_mapping = {idx: node for node, idx in node_mapping.items()}

myList = [[] for _ in range(V)]
for u, v, data in G.edges(data=True):
    weight = data.get("length", 1)  # 거리 또는 가중치
    myList[node_mapping[u]].append((node_mapping[v], weight))
    myList[node_mapping[v]].append((node_mapping[u], weight))  # 양방향 그래프

# 시작점과 도착점 설정
start_lat, start_lon = 37.864135, 127.723917  # 남춘천역
end_lat, end_lon = 37.881307, 127.730371  # 춘천시청
K = node_mapping[ox.distance.nearest_nodes(G, start_lon, start_lat)]
end_node = node_mapping[ox.distance.nearest_nodes(G, end_lon, end_lat)]

In [3]:
# 다익스트라 알고리즘
distance = [sys.maxsize] * V
visited = [False] * V
q = PriorityQueue()

#여기 빈 곳 채우기

# 최단 경로 복원
path = []
current = end_node
while current != K:
    path.append(current)
    for prev, value in myList[current]:
        if distance[prev] + value == distance[current]:
            current = prev
            break
path.append(K)
path.reverse()

In [4]:
# 결과 출력
#print("최단 경로:", [reverse_mapping[node] for node in path])
print(f"최단 경로 거리: {distance[end_node]:.2f} m")

# 시각화
# Folium 지도 생성
start_coords = (start_lat, start_lon)
m = folium.Map(location=start_coords, zoom_start=14)

# 최단 경로의 좌표 추출
route_coords = [(G.nodes[reverse_mapping[node]]["y"], G.nodes[reverse_mapping[node]]["x"]) for node in path]

# 경로를 지도에 추가
folium.PolyLine(route_coords, color="blue", weight=5, opacity=0.7).add_to(m)

# 시작점과 끝점 마커 추가
folium.Marker(location=(G.nodes[reverse_mapping[K]]["y"], G.nodes[reverse_mapping[K]]["x"]), popup="Start", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(location=(G.nodes[reverse_mapping[end_node]]["y"], G.nodes[reverse_mapping[end_node]]["x"]), popup="End", icon=folium.Icon(color="red")).add_to(m)

# 지도 저장 및 출력
m.save("shortest_path_map.html")
print("지도가 'shortest_path_map.html'로 저장되었습니다. 브라우저에서 열어보세요!")


최단 경로 거리: 2419.57 m
지도가 'shortest_path_map.html'로 저장되었습니다. 브라우저에서 열어보세요!
